# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Откройте и изучите файл

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier 
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report

In [2]:
try:
    data = pd.read_csv('/Volumes/GoogleDrive/My Drive/YP/users_behavior.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/users_behavior.csv')

In [3]:
data.head(10)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


Целевой признак - `is_ultra`. Так как значение может иметь два состояния - `1` и `0`, можно заключить, что перед нами стоит задача бинарной классификации. 

## Разбейте данные на выборки

Разделим выборку датасет на две части. Первая часть будет содержать в себе все признаки - `features`, во второй части будет хранится целевое значение - `target`.

In [5]:
features = data.drop('is_ultra',axis=1)
target = data['is_ultra']

Проверим результат:

In [6]:
features.tail()

,calls,minutes,messages,mb_used
3209,122.0,910.98,20.0,35124.90
3210,25.0,190.36,0.0,3275.61
3211,97.0,634.44,70.0,13974.06
3212,64.0,462.32,90.0,31239.78
3213,80.0,566.09,6.0,29480.52


In [7]:
target.tail()

3209    1
3210    0
3211    0
3212    0
3213    1
Name: is_ultra, dtype: int64

Теперь разделим имеющиеся выборки на три части:

1. Обучающая выборка - `60%`;
2. Валидационная выборка - `20%`;
3. Тестовая выборка - `20%`.

In [8]:
features_train, features_valid_test, target_train, target_valid_test = (
    train_test_split(features, 
                     target, 
                     test_size=.4, 
                     random_state=12345)
 )
features_valid, features_test, target_valid, target_test = (
    train_test_split(features_valid_test,
                     target_valid_test, 
                     test_size=.5, 
                     random_state=12345)
)

In [9]:
lst_shapes = [features_train.shape[0],features_valid.shape[0],features_test.shape[0]]
print(f'Датасет имеет размер в {data.shape[0]} строк' + '\n')
for value in lst_shapes:
    print('Обучающая выборка состоит из {} строк. Процентное соотношение обучающей выборки к полной: {:.0%}'
          .format(value,value/data.shape[0]))
    print('*' * 100)

Датасет имеет размер в 3214 строк

Обучающая выборка состоит из 1928 строк. Процентное соотношение обучающей выборки к полной: 60%
****************************************************************************************************
Обучающая выборка состоит из 643 строк. Процентное соотношение обучающей выборки к полной: 20%
****************************************************************************************************
Обучающая выборка состоит из 643 строк. Процентное соотношение обучающей выборки к полной: 20%
****************************************************************************************************


В результате мы имеем:

1. `features_train` и `target_train` - обучающая выборка из 1928 строк;
2. `features_valid` и `target_valid` - валидационная выборка из 643 строк;
3. `features_test` и `target_test` - тестовая выборка из 643 строк.

## Исследуйте модели

In [10]:
#Создадим словарь лучших моделей
dict_best_model = {}

Исходя из задачи выберем модели классификации:

### DecisionTreeClassifier

In [11]:
best_model_decision_tree_classifier = None
best_result_decision_tree_classifier = 0
best_depth_decision_tree_classifier = 0
for depth in range(1, 30):
	model = DecisionTreeClassifier(random_state=12345, max_depth=depth) 
	model.fit(features_train,target_train) 
	predictions = model.predict(features_valid)
	result = accuracy_score(target_valid, predictions)
	if result > best_result_decision_tree_classifier:
		best_model_decision_tree_classifier = model
		best_result_decision_tree_classifier = result
		best_depth_decision_tree_classifier = depth


dict_best_model['DecisionTreeClassifier'] = best_model_decision_tree_classifier
print("Accuracy наилучшей модели с глубиной дерева {} на валидационной выборке: {:.3}"
      .format(best_depth_decision_tree_classifier,
              best_result_decision_tree_classifier))

Accuracy наилучшей модели с глубиной дерева 3 на валидационной выборке: 0.785


### RandomForestClassifier

In [12]:
best_model_random_forest_classifier = None
best_result_random_forest_classifier  = 0
for est in range(1, 30):
    model = RandomForestClassifier(random_state=12345, n_estimators=est)
    model.fit(features_train, target_train)
    predictions = model.predict(features_valid)
    result = accuracy_score(target_valid,predictions) 
    if result > best_result_random_forest_classifier :
        best_model_random_forest_classifier  = model 
        best_result_random_forest_classifier  = result
        best_est_random_forest_classifier  = est

dict_best_model['RandomForestClassifier'] = best_model_random_forest_classifier
print("Accuracy наилучшей модели с количеством деревьев равным {} на валидационной выборке: {:.3}"
      .format(best_est_random_forest_classifier,
              best_result_random_forest_classifier))

Accuracy наилучшей модели с количеством деревьев равным 23 на валидационной выборке: 0.795


### LogisticRegression

In [13]:
best_model_logistic_regression = None
best_result_random_logistic_regression  = 0
for iter in range(100, 2500, 100):
    model = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=iter)
    model.fit(features_train, target_train)
    predictions = model.predict(features_valid)
    result = accuracy_score(target_valid,predictions)
    if result > best_result_random_logistic_regression:
        best_model_logistic_regression = model
        best_result_random_logistic_regression = result
        best_iter = iter
 
dict_best_model['LogisticRegression'] = best_model_logistic_regression        
print("Accuracy наилучшей модели с количеством итераций {} на валидационной выборке: {:.3}"
      .format(best_iter,
              best_result_random_logistic_regression))

Accuracy наилучшей модели с количеством итераций 100 на валидационной выборке: 0.711


## Проверьте модель на тестовой выборке

In [14]:
for key in dict_best_model:
    predict = dict_best_model[key].predict(features_test)
    result = accuracy_score(target_test, predict)
    print('Accuracy наилучшей модели {} на тестовых данных: {:.3}'.format(key,result))
    print('*' * 100) 

Accuracy наилучшей модели DecisionTreeClassifier на тестовых данных: 0.779
****************************************************************************************************
Accuracy наилучшей модели RandomForestClassifier на тестовых данных: 0.781
****************************************************************************************************
Accuracy наилучшей модели LogisticRegression на тестовых данных: 0.684
****************************************************************************************************


Целевой показатель (выше 0.75) Accuracy был достигнут для двух моделей - `DecisionTreeClassifier` и `RandomForestClassifier`. 

При этом, не смотря на то, что `RandomForestCLassifier` показал `Accuracy` выше, разница между тестовым значением и валидационным составляет: `0.014`. В то время как у `DecisionTreeClassifier` это значение меньше: `0.06`. 

Стоит учитывать это при выборе модели. Однако, если произвести интерполяцию, можно сделать вывод, что точность `RandomForestCLassifier` при прогнозирование данных окажется выше. 

In [15]:
report = classification_report(target_test, 
                               dict_best_model['RandomForestClassifier'].predict(features_test), 
                               target_names=['is_smart', 'is_ultra'])
print(report)

              precision    recall  f1-score   support

    is_smart       0.82      0.87      0.84       440
    is_ultra       0.68      0.58      0.63       203

    accuracy                           0.78       643
   macro avg       0.75      0.73      0.74       643
weighted avg       0.77      0.78      0.78       643



Нас интересует `f1-score`, которая представляет собой среднее гармоническое между precision и recall. Значение метрики - `0.63`, что свидетельствует о среднем качестве модели. 

## Вывод

В исследовании был проведен анализ архива тарифов оператора сотовой связи. 

a. Исходные данные:

Исход данные представляли собой датасет с размером: 3214 строки и 5 колонок.

b. Разделение данных: 

Исходя из поставленной задачи данные были разделены на целевой показатель `is_ultra` и признаки: `calls`,`minutes`,`messages`,`mb_used`;

После `features` и `target` были разделены на три выборки: `train`, `valid`, `test`.

c. Проведено обучение и оценены модели с разными гиперпараметрами;

d. Модели были протестированы на тестовых данных. Наилучшей моделью оказалась `RandomForestClassifier` c `n_estimators` равным `23`. `Accuracy` составило `0.78`.